In [1]:
import cv2

In [2]:
import numpy as np
import os
import math

In [5]:
cap=cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")


skip=0
face_data = []
dataset_path='./data/'

file_name=input("Enter the name of person :")

while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
    
    grey_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    faces = sorted(faces,key=lambda f:f[2]*f[3],reverse=True)
    
    for face in faces:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        #ectract region of interest
        offset =10
        face_section =frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        skip+=1
        if skip%10==0:
            face_data.append(face_section)
            print(len(face_data))
        
        
        
       
            
        
    cv2.imshow("Frame",frame)
    cv2.imshow("face section",face_section)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
            break
cap.release()        

cv2.destroyAllWindows()        
    

Enter the name of person :harsh
1
2
3
4
5
6


In [6]:
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

#save data to file system
np.save(dataset_path+file_name+'.npy',face_data)
print("Data Succesfully save at "+dataset_path+file_name+'.npy')

(6, 30000)
Data Succesfully save at ./data/harsh.npy


In [3]:
#building face classifier
def distance(x1, x2) :
    return math.sqrt(np.sum((x1-x2) ** 2))

def knn(train, test, k=5):
    
    dist = []
    
    for i in range(train.shape[0]):
        ix = train[i,:-1]
        iy = train[i,-1]
        
        d=distance(test,ix)
        dist.append([d,iy])
        
    dk = sorted(dist,key=lambda x:x[0])[:k]
    labels=np.array(dk)[:,-1]
    
    output= np.unique(labels,return_counts=True)
    index=np.argmax(output[1])
    return output[0][index]
    






In [4]:


cap=cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
face_data2 = []
dataset_path='./data/'


labels=[]

class_id=0  #label for the given file
names ={} #mapping btw id - name


#data preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]
        print("Loaded " + fx)
        
        data_item = np.load(dataset_path+fx)
        face_data2.append(data_item)
        
        #create labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id=+1
        labels.append(target)
        
        
    
face_dataset = np.concatenate(face_data2,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)



while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
    

    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    
    for face in faces:
        x,y,w,h = face
        
        #ectract region of interest
        offset =10
        face_section =frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        out = knn(trainset,face_section.flatten()) #PREDICT TABLE
        
        
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
           
        
    cv2.imshow("Frame",frame)
    
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
            break
cap.release()        

cv2.destroyAllWindows()   

Loaded danish.npy
Loaded harsh.npy
Loaded priyanka.npy
(30, 30000)
(30, 1)
